In [1]:
from tqdm import tqdm
from scipy.io import savemat
from Gantry.envs.GantrySimulation import GantrySimulation
import torch
import time
import pybullet as p
import pybullet_data
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from Gantry.controller.SNS_layer import perceptor, controller_open_loop, controller_modulation, controller_closed_loop_modulation, pick_and_place, sensory_layer_2_feedback_modulation, SNS_Control_closed_loop_v1, SNS_Control_closed_loop_v2, SNS_Control_closed_loop_modulation, controller_closed_loop_v2, R, perceptor_closed_loop_modulation

alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!


c:\Users\Yanjun\Documents\MATLAB\FRR_SoftGrasper\PyBullet Model\Gantry\controller\SNS_layer.py:396: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sensory_layer_2_param = tor

## Some constant parameters used in simulation.

object_position = [0, 0, -0.34] (m)

target_position = [0.15, 0.15, -0.34] (m)


size scaling factor = 0.6

simulation end time = 15 (sec)

open-loop perceptor = perceptor (with an inhibitory synapse from the grasping-the-object command neuron to the lifting-the-object-up command neuron. This additional synapse can prevent the controller from transitioning to the lifting up phase too early)

closed-loop perceptor = perceptor_closed_loop_modulation (similar to the open-loop perceptor except that the force thresholds can be modulated)

open-loop controller with time history dependent control = controller_open_loop (jaws use position control)

closed-loop controller = controller_closed_loop_v2 (jaws use speed control to regulate force)

force threshold gain = 0.1 (Initially multiplying force sensory signals by 10. Warning: this method may only work in simulation. Using a small gain in the experiment may lead to a force threshold lower than the noise level)

jaw pressure used in the open loop controller = 2.5 (PSI)

grasper closing speed gain = 0.3 

time constants of neurons in the perceptor = 0.05 (sec)

In [3]:
EXPERIMENT_NUM = int(3)
OBJECT_NUM = int(20)

OBJECT_POSITION = [0, 0, -0.34]
TARGET_POSITION = [0.15, 0.15, -0.34]
SIZE_SCALING = 0.6

END_TIME = 8

PERCEPTOR_TAU = 0.1
FORCE_MODULATION_GAIN = 5

OPEN_LOOP_PERCEPTOR = perceptor
OPEN_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)
CLOSED_LOOP_PERCEPTOR = perceptor
CLOSED_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)

OPEN_LOOP_CONTROLLER = controller_open_loop
CLOSED_LOOP_CONTROLLER = controller_closed_loop_v2

FORCE_THRESHOLD_GAIN = 0.1
PRESSURE_VALUE = 2.5
GRASPER_CLOSING_SPEED = 0.3

## Simulation settings

In [ ]:
def mass_generate(object_num, experiment_num, light_object_range, heavy_object_range):
    mass_set = np.zeros((experiment_num, 11, object_num))
    for group_index, heavy_proportion in enumerate(np.linspace(0, 1, 11)):
        heavy_num = int(heavy_proportion * object_num)
        mass_set[:, int(group_index), :heavy_num] = np.random.uniform(heavy_object_range[0], heavy_object_range[1], (experiment_num, heavy_num))
        mass_set[:, int(group_index), heavy_num:object_num + 1] = np.random.uniform(light_object_range[0], light_object_range[1], (experiment_num, object_num - heavy_num))
    
    return mass_set

def pick_and_place_experiment(mass_set, perceptor, controller, experiment_num=EXPERIMENT_NUM, object_num=OBJECT_NUM, position_o=OBJECT_POSITION, position_t=TARGET_POSITION, end_time=END_TIME, force_threshold_gain=FORCE_THRESHOLD_GAIN, grasper_closing_speed=GRASPER_CLOSING_SPEED, sizeScaling=SIZE_SCALING, video_logging=False, early_stop=True):
    F_max = np.zeros((experiment_num, 11, object_num))
    success = np.zeros((experiment_num, 11, object_num))

    for experiment_index in range(experiment_num):
        for group_index in range(11):
            for object_index in tqdm(range(object_num), desc=f"Experinment {experiment_index}, Group {group_index}", leave=False):
                mass = mass_set[experiment_index, group_index, object_index]
                perceptor.reset()
                controller.reset()
                neuron_history, sensory_history, _, _ = pick_and_place(position_o=position_o, position_t=position_t, perceptor=perceptor, controller=controller, end_time=end_time, force_threshold_gain=force_threshold_gain * (3 + group_index / 10), grasper_closing_speed=grasper_closing_speed, zero_time_constant=True, mass=mass, sizeScaling=sizeScaling, video_logging=video_logging, early_stop=early_stop)

                F_max[experiment_index, group_index, object_index] = np.max(np.sum(sensory_history[:, 3:6], axis=1))
                if np.max(neuron_history[:, -1]) > 10:
                    success[experiment_index, group_index, object_index] = 1
    
    return F_max, success

## Control simulation

We test the performance of two pick-and-place controllers when noise is injected into sensory signals (one with nonzero time constants and another with zero time constants).

In [4]:
mass = 0.2

PERCEPTOR_TAU = 0.05

OPEN_LOOP_PERCEPTOR = perceptor
OPEN_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)
CLOSED_LOOP_PERCEPTOR = perceptor
CLOSED_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)

OPEN_LOOP_PERCEPTOR.reset()
OPEN_LOOP_CONTROLLER.reset()
neuron_history, sensory_history, command_history, inter_history, trajectory = pick_and_place(position_o=OBJECT_POSITION, position_t=TARGET_POSITION, perceptor=OPEN_LOOP_PERCEPTOR, controller=OPEN_LOOP_CONTROLLER, end_time=8, force_threshold_gain=FORCE_THRESHOLD_GAIN, grasper_closing_speed=GRASPER_CLOSING_SPEED, zero_time_constant=True, mass=mass, sizeScaling=SIZE_SCALING, video_logging=True, cameraDistance=0.6, cameraYaw=0, cameraPitch=-10, cameraTargetPosition=[0, 0, 0.1], early_stop=True, noise_level=0.01)

non_zero_tau = {"neuron_history": neuron_history, "sensory_history": sensory_history, "command_history": command_history, "trajectory": trajectory}
savemat("non_zero_tau.mat", non_zero_tau)


PERCEPTOR_TAU = 0.0

OPEN_LOOP_PERCEPTOR = perceptor
OPEN_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)
CLOSED_LOOP_PERCEPTOR = perceptor
CLOSED_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)

OPEN_LOOP_PERCEPTOR.reset()
OPEN_LOOP_CONTROLLER.reset()
neuron_history, sensory_history, command_history, inter_history, trajectory = pick_and_place(position_o=OBJECT_POSITION, position_t=TARGET_POSITION, perceptor=OPEN_LOOP_PERCEPTOR, controller=OPEN_LOOP_CONTROLLER, end_time=8, force_threshold_gain=FORCE_THRESHOLD_GAIN, grasper_closing_speed=GRASPER_CLOSING_SPEED, zero_time_constant=True, mass=mass, sizeScaling=SIZE_SCALING, video_logging=True, cameraDistance=0.6, cameraYaw=0, cameraPitch=-10, cameraTargetPosition=[0, 0, 0.1], early_stop=True, noise_level=0.01)

zero_tau = {"neuron_history": neuron_history, "sensory_history": sensory_history, "command_history": command_history, "trajectory": trajectory}
savemat("zero_tau.mat", zero_tau)
